In [ ]:

from osgeo import gdal
import numpy as np
import rasterio
import cv2
#img = gdal.Open('F:/Pedro/Facul/TCC/Dataset/RGBNIR_CHM.tif').ReadAsArray()
#
#print(img.shape)
##img = np.swapaxes(img,0,2)
#print(img.shape)
#
#img2 = np.ones((img.shape[1],img.shape[2]))

with rasterio.open(f"F:/Pedro/Facul/TCC/Dataset/RGBNIR_CHM.tif") as src:
    img_5ch = src.read()
    meta = src.meta  # Preserve the metadata for later use,
    img_3ch = np.ones((3,img_5ch.shape[1],img_5ch.shape[2]))
    img_3ch[2] = img_5ch[2].copy()
    img_3ch[1] = img_5ch[1].copy()
    img_3ch[0] = img_5ch[0].copy()


    # count indica pro rasterio quantos canais tem a imagem, se tirar o dst.write da erro pq o 'count' original era 5
    meta['count'] = 3
    # Update the data type to uint8
    meta['dtype'] = 'uint8'
    #(4-3)/(4+3)img[3]
    img2 = (abs(img_5ch[3]-img_5ch[2]))/(img_5ch[3]+img_5ch[2])
    trees_pos = np.where(img2>0.3)
    non_trees_pos = np.where(img2<=0.3)
    


    print(len(trees_pos))
    print(len(trees_pos[0]))
    print(len(trees_pos[1]))
    print(f"img2.width = {img2.shape[0]}")
    print(f"img2.height = {img2.shape[1]}")
    print(f"img2.pixel_count = {img2.shape[0]*img2.shape[1]}")
    print(f"trees_pos.size = {len(trees_pos[0])}")
    print(f"non_trees_pos.size = {len(non_trees_pos[0])}")
    print(f"trees_pos.size + non_trees_pos.size= {len(trees_pos[0])+len(non_trees_pos[0])}")
    print(f"diff= {len(trees_pos[0])+len(non_trees_pos[0])-(img2.shape[0]*img2.shape[1])}")

    print(f"trees_pos.x.max = {(trees_pos[0][-1])}")
    print(f"trees_pos.y.max = {(trees_pos[1][-1])}")
    with rasterio.open(f'F:/Pedro/Facul/TCC/Dataset/RGB_3ch.png', 'w', **meta) as dst:
        dst.write(img_3ch)

#img = np.swapaxes(img2,0,2)



In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit    
from sklearn import datasets

img_list = ["0_0.png","0_1.png","0_2.png","0_3.png","0_4.png"]
label_name_list = ["0_10","9_20","11_18","7_8","16_24"]

label_list =[]

for label_name in label_name_list:
    with open(f"F:/Pedro/YOLOv8/My_Code/datasets/full/labels/{label_name}.txt") as file:
        Lines = file.readlines()
        bb_list = []
        count = 0
        # Strips the newline character
        print(f"There are {len(Lines)} bounding boxes in this image.")
        color = 0
        for line in Lines:
            classe = line.strip().split()[0]
            bb = line.strip().split()[1:]
            #print(classe,bb)
            ycenter = float(bb[0])
            xcenter = float(bb[1])
            height   = float(bb[2])
            width  = float(bb[3])
            xmin = xcenter - (width/2)
            ymin = ycenter - (height/2)
            xmax = xcenter + (width/2)
            ymax = ycenter + (height/2)


            bb_list.append([int(classe)])
        label_list.append(bb_list)
print(label_list)
train_samples = 3
test_samples = 2
total_samples = test_samples + train_samples
X, y = datasets.make_blobs(n_samples=total_samples, centers=2, random_state=0)
print(f"X = {X}")
print(f"y = {y}")

sss = StratifiedShuffleSplit(train_size=train_samples, n_splits=1, 
                             test_size=test_samples, random_state=0)  
#"""
for train_index, test_index in sss.split(label_list, label_name_list):
    print(f"train_index = {train_index}")
    print(f"test_index = {test_index}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#"""

In [11]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

# Example YOLO format labels for multiple images
yolo_labels_multiple_images = [
    [
        [0, [0.3, 0.4, 0.5, 0.6]],  # Class 0
        [1, [0.1, 0.2, 0.3, 0.4]],  # Class 1
    ],
    [
        [0, [0.2, 0.3, 0.4, 0.5]],  # Class 0
        [2, [0.4, 0.5, 0.6, 0.7]],  # Class 2
    ],
    [
        [1, [0.5, 0.6, 0.7, 0.8]],  # Class 1
        [2, [0.1, 0.2, 0.3, 0.4]],  # Class 2
    ]
]

# Extract class labels from YOLO format labels for each image
labels_per_image = [np.array([label[0] for label in img_labels]) for img_labels in yolo_labels_multiple_images]

# Concatenate the class labels for all images into a single array
labels = np.concatenate(labels_per_image)

# Create corresponding indices to identify each image's label
image_indices = np.concatenate([np.full(len(img_labels), i) for i, img_labels in enumerate(yolo_labels_multiple_images)])

# Create the StratifiedShuffleSplit object
sss = StratifiedShuffleSplit(n_splits=1, test_size=3, random_state=42)
print(labels)
# Use split method with labels and image indices
"""
for train_index, test_index in sss.split(np.zeros((len(labels), 1)), labels, image_indices):
    X_train, X_test = np.array(yolo_labels_multiple_images)[train_index], np.array(yolo_labels_multiple_images)[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
#"""
    # Here, X_train and y_train will contain the training data (YOLO format) and corresponding labels
    # X_test and y_test will contain the test data (YOLO format) and corresponding labels for each split.


[0 1 0 2 1 2]


'\nfor train_index, test_index in sss.split(np.zeros((len(labels), 1)), labels, image_indices):\n    X_train, X_test = np.array(yolo_labels_multiple_images)[train_index], np.array(yolo_labels_multiple_images)[test_index]\n    y_train, y_test = labels[train_index], labels[test_index]\n#'